# Introduction to Memory Profiling

> Objectives:
> * Be introduced to memory profiling using different tools
> * Some small introduction to time profiling in IPython too


## ipython_memwatcher

Our recommended way to profile memory consumption for this tutorial will be [ipython_memwatcher](https://pypi.python.org/pypi/ipython_memwatcher):


In [2]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [2] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 44.805 MiB


In [3]:
# Let's create a big object
a = [i for i in range(1000*1000)]

In [3] used 39.426 MiB RAM in 0.077s, peaked 0.000 MiB above current, total RAM usage 84.230 MiB


In [4]:
# Get some measurements from the last executed cell:
meas = mw.measurements
meas

Measurements(memory_delta=39.42578125, time_delta=0.07696342468261719, memory_peak=0, memory_usage=84.23046875)

In [4] used 0.051 MiB RAM in 0.023s, peaked 0.000 MiB above current, total RAM usage 84.281 MiB


In [5]:
# MemWatcher.measurements is a named tuple.  We can easily get info out of it:
meas.memory_delta

39.42578125

In [5] used 0.000 MiB RAM in 0.005s, peaked 0.000 MiB above current, total RAM usage 84.281 MiB


In [6]:
# This takes between 32 ~ 35 bytes per element:
meas.memory_delta * (2**20) / 1e6

41.340928

In [6] used 0.000 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 84.281 MiB


In [7]:
# What are these elements made of?
type(a[0])

int

In [7] used 0.000 MiB RAM in 0.009s, peaked 0.000 MiB above current, total RAM usage 84.281 MiB


In [8]:
# How much memory take an int?
# Beware: the size below will depend on whether you are using a 32-bit or 64-bit Python
import sys
sys.getsizeof(a[0])

24

In [8] used 0.000 MiB RAM in 0.009s, peaked 0.000 MiB above current, total RAM usage 84.281 MiB


Ok.  On 64-bits, that means that the int object allocates 8 bytes for the integer value, and 16 bytes for other metadata (Python object).  But 24 is quite less than 32~35.  Where this overhead comes from?

Well, it turns out that the list structure needs additional pointers to reference the different components.

![List diagram](http://www.brpreiss.com/books/opus7/html/img579.gif)

## memory_profiler

[memory_profiler](https://pypi.python.org/pypi/memory_profiler) is a basic module for memory profiling that many others use (like the `ipython_memwatcher` above) and it interacts well with ipython, so it is worth to see how it works:

In [9]:
%load_ext memory_profiler

In [9] used 0.000 MiB RAM in 0.012s, peaked 0.000 MiB above current, total RAM usage 84.281 MiB


In [10]:
# Use %memit magic command exposed by memory_profiler
%memit b = [i for i in range(1000*1000)]

peak memory: 157.75 MiB, increment: 73.40 MiB
In [10] used 39.836 MiB RAM in 0.329s, peaked 0.000 MiB above current, total RAM usage 124.117 MiB


Please note that the `peak_memory` in this case is different than the `peaked_memory` reported by ipython_memwatcher package.

## %time and %timeit

In [12]:
# IPython provides a magic command to see how much time a command takes to run
%time asum = sum(a)

CPU times: user 24 ms, sys: 12 ms, total: 36 ms
Wall time: 37.4 ms
In [12] used 0.000 MiB RAM in 0.043s, peaked 0.000 MiB above current, total RAM usage 124.352 MiB


Note that `%time` offers quite detailed statistics on the time spent.

Also, the time reported by MemoryWatcher has a typical overhead of 1~5 ms over the time reported by %time, so when the times to measure are about this order then it is better to rely on the %time (or %timeit below) values.  

In [13]:
# We have another way to measure timings doing several loops and getting the mean
%timeit bsum = sum(a)
# However, one must notice that %timeit does not return the result of expressions

100 loops, best of 3: 9.22 ms per loop
In [13] used 0.000 MiB RAM in 4.007s, peaked 0.000 MiB above current, total RAM usage 124.352 MiB


Interestingly, %timeit allows to retrieve the measured times in loops with the -o flag:

In [14]:
t = %timeit -o sum(a)
print(t.all_runs)
print(t.best)

100 loops, best of 3: 9.2 ms per loop
[0.9200300000084098, 0.9270500400016317, 1.1027420629980043]
0.0092003000000841
In [14] used 0.000 MiB RAM in 4.069s, peaked 0.000 MiB above current, total RAM usage 124.352 MiB


And one can specify the number of loops (-n) and the number of repetitions (-r):

In [15]:
t = %timeit -r1 -n1 -o sum(a)
print(t.all_runs)
print(t.best)

1 loop, best of 1: 27.3 ms per loop
[0.027260237009613775]
0.027260237009613775
In [15] used 0.000 MiB RAM in 0.037s, peaked 0.000 MiB above current, total RAM usage 124.352 MiB


### Exercise 1

Provided a dictionary like:

```
d = dict(("key: %i"%i, i*2) for i in a)
```

Try to guess how much RAM it uses using the techniques introduced above.

Why do you think it takes more space than a list?

*Hint*: Every entry in a dictionary has pointers to two objects: key and value. 

## Solution

In [16]:
d = dict(("key: %i"%i, i*2) for i in a)

In [16] used 151.270 MiB RAM in 0.716s, peaked 0.000 MiB above current, total RAM usage 275.621 MiB


In [17]:
# Compute the size of key + value
sys.getsizeof("key: 100000") + sys.getsizeof(1)

88

In [17] used 0.023 MiB RAM in 0.007s, peaked 0.000 MiB above current, total RAM usage 275.645 MiB


So, `sys.getsizeof()` is telling us that the keys + values take ~84 MB.  However, our `MemWatcher` instance is reporting just ~150 MB, so that means that the dict structure takes around ~66 MB.  The take away lesson is: do not assume that the consumption is just in the elements; the data container can also take quite a bit of memory!